In [1]:
spark

In [2]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from datetime import datetime, timedelta
import argparse

# Edges

##### Nacteni dat

In [6]:
game_plays = sqlContext.read.option("header", "true").option("delimiter", ",").csv("skola")

##### Filtrace play_id, ve kterych padl gol a sezony

In [12]:
goals = game_plays.select('play_id').where("playerType = 'Scorer'").where("game_id > 2017090000")

In [160]:
goals.show(3)

+--------------+
|       play_id|
+--------------+
| 2018020001_44|
| 2018020001_68|
|2018020001_213|
+--------------+
only showing top 3 rows



##### Pravy join na game_plays

In [31]:
goals_goalie = game_plays.join(goals, game_plays["play_id"] == goals["play_id"], how='right').withColumn('play_id_player',game_plays['play_id']).withColumn('play_id_goalie',game_plays['play_id'])

In [161]:
goals_goalie.show(5)

+--------------+----------+--------+---------+----------+--------------+--------------+--------------+
|       play_id|   game_id|play_num|player_id|playerType|       play_id|play_id_player|play_id_goalie|
+--------------+----------+--------+---------+----------+--------------+--------------+--------------+
|2018020014_261|2018020014|     261|  8477930|    Scorer|2018020014_261|2018020014_261|2018020014_261|
|2018020014_261|2018020014|     261|  8470880|    Goalie|2018020014_261|2018020014_261|2018020014_261|
|  2018020020_9|2018020020|       9|  8475151|    Scorer|  2018020020_9|  2018020020_9|  2018020020_9|
|  2018020020_9|2018020020|       9|  8476923|    Assist|  2018020020_9|  2018020020_9|  2018020020_9|
|  2018020020_9|2018020020|       9|  8475660|    Goalie|  2018020020_9|  2018020020_9|  2018020020_9|
+--------------+----------+--------+---------+----------+--------------+--------------+--------------+
only showing top 5 rows



In [62]:
players = goals_goalie.where("playerType = 'Scorer'").select('play_id_player',F.col('game_id').alias('game_id_player'),F.col('player_id').alias('player_id_player'),F.col('playerType').alias('playerTypePlayer'))

In [63]:
goalies = goals_goalie.where("playerType = 'Goalie'").select('play_id_goalie',F.col('game_id').alias('game_id_goalie'),F.col('player_id').alias('player_id_goalie'),F.col('playerType').alias('playerTypeGoalie'))

##### Inner join hracu a golmanu - zmena struktury

In [67]:
df = players.join(goalies,players["play_id_player"] == goalies["play_id_goalie"])

In [162]:
df.show(10)

+--------------+--------------+----------------+----------------+--------------+--------------+----------------+----------------+
|play_id_player|game_id_player|player_id_player|playerTypePlayer|play_id_goalie|game_id_goalie|player_id_goalie|playerTypeGoalie|
+--------------+--------------+----------------+----------------+--------------+--------------+----------------+----------------+
|2018020014_261|    2018020014|         8477930|          Scorer|2018020014_261|    2018020014|         8470880|          Goalie|
|  2018020020_9|    2018020020|         8475151|          Scorer|  2018020020_9|    2018020020|         8475660|          Goalie|
|2018020084_115|    2018020084|         8476460|          Scorer|2018020084_115|    2018020084|         8475660|          Goalie|
|2018020089_196|    2018020089|         8474641|          Scorer|2018020089_196|    2018020089|         8471306|          Goalie|
|2018020092_128|    2018020092|         8474715|          Scorer|2018020092_128|    201802

##### Vytvoreni vztahu hrac -> golman, pocet golu

In [94]:
edges = df.groupBy('player_id_player','player_id_goalie').agg(F.count('*').alias('Goals'))

In [148]:
final_edges = edges.select(F.col('player_id_player').alias('Source'),F.col('player_id_goalie').alias('Target'),'Goals').withColumn('Relationship',F.lit('GOAL'))

In [164]:
final_edges.show(10)

+-------+-------+-----+------------+
| Source| Target|Goals|Relationship|
+-------+-------+-----+------------+
|8478366|8474651|    1|        GOAL|
|8474884|8476876|    2|        GOAL|
|8477511|8470860|    3|        GOAL|
|8479337|8476343|    2|        GOAL|
|8479395|8476914|    1|        GOAL|
|8474679|8471306|    1|        GOAL|
|8470047|8475195|    2|        GOAL|
|8471709|8475831|    2|        GOAL|
|8474009|8473575|    1|        GOAL|
|8468508|8475195|    2|        GOAL|
+-------+-------+-----+------------+
only showing top 10 rows



##### Export hran

In [151]:
final_edges.repartition(1).write.csv('nhl_edges.csv',sep=",")

# Nodes

### Nacteni dat - game_plays, player_info, team_info

In [100]:
plays = sqlContext.read.option("header", "true").option("delimiter", ",").csv("game_plays.csv")

In [110]:
player_info = sqlContext.read.option("header", "true").option("delimiter", ",").csv("player_info.csv")

In [130]:
team_info = sqlContext.read.option("header", "true").option("delimiter", ",").csv("team_info.csv")

#### filtrace sezony

In [102]:
plays_filtered = plays.where("game_id > 2017090000")

#### nacteni hracu a golmanu z goals_goalie definovanych u hran

In [117]:
players_list = goals_goalie.where("playerType IN ('Scorer','Goalie')").select('play_id_player','player_id','playerType')

In [166]:
players_list.show(5)

+--------------+---------+----------+
|play_id_player|player_id|playerType|
+--------------+---------+----------+
|2018020014_261|  8477930|    Scorer|
|2018020014_261|  8470880|    Goalie|
|  2018020020_9|  8475151|    Scorer|
|  2018020020_9|  8475660|    Goalie|
|2018020084_115|  8476460|    Scorer|
+--------------+---------+----------+
only showing top 5 rows



#### pravy orez game_plays pouze na strelce a golmany podle play_id

In [118]:
players_teams = plays_filtered.join(players_list, plays_filtered["play_id"] == players_list["play_id_player"], how = 'right')

In [167]:
players_teams.show(10)

+--------------+----------+--------+-----------+---------------+-----+-------------+---+---+------+----------+----------+-------------------+-------------------+----------+----------+--------------------+----+----+---------+--------------+---------+----------+
|       play_id|   game_id|play_num|team_id_for|team_id_against|event|secondaryType|  x|  y|period|periodType|periodTime|periodTimeRemaining|           dateTime|goals_away|goals_home|         description|st_x|st_y|rink_side|play_id_player|player_id|playerType|
+--------------+----------+--------+-----------+---------------+-----+-------------+---+---+------+----------+----------+-------------------+-------------------+----------+----------+--------------------+----+----+---------+--------------+---------+----------+
|2018020014_261|2018020014|     261|         54|              4| Goal|   Wrist Shot|-82| -2|     3|   REGULAR|       508|                692|2018-10-05 04:19:53|         5|         2|Pierre-Edouard Be...|  82|   2|   

#### prirazeni kodu tymu jednotlivym hracum podle play_id - zde se muze stat, ze se hraci zmei tym po prestupu

In [121]:
player_ids = players_teams.withColumn('team',F.expr("CASE WHEN playerType = 'Scorer' THEN team_id_for WHEN playerType = 'Goalie' THEN team_id_against END "))

In [155]:
player_team = player_ids.select('team',F.col('player_id').alias('player'),'playerType')

#### join id hrace s detaily o hraci

In [156]:
player_complete = player_team.join(player_info, player_team["player"] == player_info["player_id"], how = 'left')

In [168]:
player_complete.show(5)

+----+-------+----------+---------+--------------+---------+-----------+---------------+---------------+----------+--------------------+
|team| player|playerType|player_id|     firstName| lastName|nationality|      birthCity|primaryPosition| birthDate|                link|
+----+-------+----------+---------+--------------+---------+-----------+---------------+---------------+----------+--------------------+
|  54|8477930|    Scorer|  8477930|Pierre-Edouard|Bellemare|        FRA|Le Blanc-Mesnil|             LW|1985-03-06|/api/v1/people/84...|
|   4|8470880|    Goalie|  8470880|         Brian|  Elliott|        CAN|      Newmarket|              G|1985-04-09|/api/v1/people/84...|
|   1|8475151|    Scorer|  8475151|          Kyle| Palmieri|        USA|      Smithtown|             RW|1991-02-01|/api/v1/people/84...|
|  22|8475660|    Goalie|  8475660|           Cam|   Talbot|        CAN|      Caledonia|              G|1987-07-05|/api/v1/people/84...|
|  52|8476460|    Scorer|  8476460|      

#### join informaci o hraci s tymem, prejmenovani atributu

In [157]:
nodes = (player_complete.join(team_info, team_info["team_id"] == player_complete["team"],how = 'left')
        .withColumn('name',F.expr("concat(firstName,' ',lastName)")).withColumn('fullTeamName',F.expr("concat(shortName,' ',teamName)"))
        .select('player_id','name','nationality','primaryPosition','playerType','birthDate','fullTeamName','abbreviation').distinct())

In [169]:
nodes.show(2)

+---------+-------------+-----------+---------------+----------+----------+--------------------+------------+
|player_id|         name|nationality|primaryPosition|playerType| birthDate|        fullTeamName|abbreviation|
+---------+-------------+-----------+---------------+----------+----------+--------------------+------------+
|  8478466|Daniel Sprong|        NLD|             RW|    Scorer|1997-03-17|       Anaheim Ducks|         ANA|
|  8477901|  Daniel Carr|        CAN|             LW|    Scorer|1991-11-01|Vegas Golden Knights|         VGK|
+---------+-------------+-----------+---------------+----------+----------+--------------------+------------+
only showing top 2 rows



#### export uzlu

In [159]:
nodes.repartition(1).write.csv('nodes',sep=",")